Genera un diccionario con todos los statutes del reto en s_dict

In [ ]:
import os

directory = "../dataset/Object_statutes"
# crea un diccionario con las sentencias en el que la clave es el nombre del fichero
# sin el .txt y el valor es el contenido del fichero
s_dict = {}
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        filepath = os.path.join(directory, filename)
        with open(filepath, "r") as file:
            content = file.read()
            key, value = filename.split(".")[0], content
            # añadir al diccionario si el value no esta vacio
            if value:
                s_dict[key] = value
            


n_statutes = len(s_dict)  # numero de sentencias
print("Numero de sentencias: ",n_statutes)


In [ ]:
directory = "../dataset/Object_casedocs"
# crea un diccionario con los casos en el que la clave es el nombre del fichero
# sin el .txt y el valor es el contenido del fichero
c_dict = {}
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        filepath = os.path.join(directory, filename)
        with open(filepath, "r") as file:
            content = file.read()
            key, value = filename.split(".")[0], content
            # añadir al diccionario si el value tiene contenido
            if value:
                c_dict[key] = value

n_cases = len(c_dict)  # numero de casos
print("Numero de casos: ",n_cases)

In [ ]:
# Crea una lista a partir del diccionario de sentencias con la forma "id": clave, "text": valor
sentences = [{"id": key, "text": value} for key, value in s_dict.items()]
# Crea una lista a partir del diccionario de casos con la forma "id": clave, "text": valor
cases = [{"id": key, "text": value} for key, value in c_dict.items()]

import json
# escribe cada una de las listas en un fichero json de sentencias 
with open("../dataset/sentences.json", "w") as file:
    json.dump(sentences, file, indent=4)

# escribe cada una de las listas en 15 ficheros de casos separados en partes iguales
# con el mismo numero de casos
n = n_cases // 15
for i in range(15):
    with open(f"../dataset/cases_{i}.json", "w") as file:
        json.dump(cases[i*n:(i+1)*n], file, indent=4)



In [ ]:
query_file = "../dataset/Query_doc.txt"
with open(query_file, "r") as file:
    query = file.read()

# a partir de query crea un diccionario dividendo por líneas. 
# La clave es la cadena antes del símbolo || y el valor lo que sigue hasta el final de la línea
query_dict = {}
for line in query.split("\n"):
    if line != "":
        key, value = line.split("||")
        query_dict[key] = value
n_queries = len(query_dict)  # numero de queries
print("Numero de queries: ",n_queries)


In [ ]:
import random
import copy

file_path = "../dataset/relevance_judgments_statutes.txt"
factor = 4 # factor de multiplicación de sentencias no relevantes

with open(file_path, "r") as file:
    content = file.read()

# crea un diccionario con las sentencias relevantes para cada query
# la clave es el nombre de la query y el valor es una lista con las sentencias relevantes
# cada línea tiene la forma "AILA_Q1 Q0 S1 1" Q0 es un separador, S1 es la sentencia y 
# 1 si es relevante, 0 cero si no lo es

s_relevance_dict = {} # diccionario con las sentencias relevantes indexadas por query
for line in content.split("\n"):
    # si line termina en un 1, entonces es relevante
    if line.endswith("1"):
        key, value = line.split(" ")[0], line.split(" ")[2]
        if key in s_relevance_dict:
            if value in s_dict: # existen statutes como solución que no existen en el dict ej S58
                s_relevance_dict[key].append(value)
        else:
            if value in s_dict: # existen statutes como solución que no existen en el dict ej S58
                s_relevance_dict[key] = [value]

# imprime por pantalla la query que tiene más sentencias relevantes y el número de sentencias relevantes
max_key = max(s_relevance_dict, key=lambda x: len(s_relevance_dict[x]))
print("Query con más sentencias relevantes: ",max_key)
print("Número de sentencias relevantes: ",len(s_relevance_dict[max_key]))


# s_query_dict es un diccionario con todas las sentencias relevantes y no relevantes
# indexadas por query, y con un número de sentencias no relevantes igual a "factor" por número
# de sentencias relevantes. Es decir, si para una query hay 3 sentencias relevantes,
# y factor es 4, entonces el número de sentencias no relevantes será 3*4 = 12.
# y el total para esa query será 15.

s_query_dict = copy.deepcopy(s_relevance_dict) # copia el diccionario de sentencias relevantes

# crea una lista con las claves de s_dict
s_keys = list(s_dict.keys())
for key in s_query_dict:
    N_statutes_check = factor * len(s_query_dict[key]) # número de sentencias no relevantes a añadir
    # baraja s_keys para no tomar siempre las mismas de relleno
    random.shuffle(s_keys)
    while len(s_query_dict[key]) < N_statutes_check:
        for s in s_keys:
            if s not in s_query_dict[key]:
                s_query_dict[key].append(s)
                if len(s_query_dict[key]) == N_statutes_check:
                    break


# baraja los elementos de la lista de valores de cada entrada de s_query_dict
# para que no estén siempre al principio las sentencias relevantes
for key in s_query_dict:
    random.shuffle(s_query_dict[key])



In [ ]:
import random
import copy

file_path = "../dataset/relevance_judgments_priorcases.txt"
factor = 4 # factor de multiplicación de casos no relevantes

with open(file_path, "r") as file:
    content = file.read()

# crea un diccionario con los casos relevantes para cada query
# la clave es el nombre de la query y el valor es una lista con las casos relevantes
# cada línea tiene la forma "AILA_Q1 Q0 C1 1" Q0 es un separador, C1 es el caso y 
# 1 si es relevante, 0 cero si no lo es

c_relevance_dict = {} # diccionario con los casos relevantes indexadas por query
for line in content.split("\n"):
    # si line termina en un 1, entonces es relevante
    if line.endswith("1"):
        key, value = line.split(" ")[0], line.split(" ")[2]
        if key in c_relevance_dict:
            if value in c_dict: # existen casos como solución que podrían no existir en el dict
                c_relevance_dict[key].append(value)
        else:
            if value in c_dict: # existen statutes como solución que no existen en el dict ej S58
                c_relevance_dict[key] = [value]


# c_query_dict es un diccionario con todos las casos relevantes y no relevantes
# indexadas por query, y con un número de casos no relevantes igual a "factor" por número
# de casos relevantes. Es decir, si para una query hay 3 casos relevantes,
# y factor es 4, entonces el número de casos no relevantes será 3*4 = 12.
# y el total para esa query será 15.

c_query_dict = copy.deepcopy(c_relevance_dict) # copia el diccionario de casos relevantes

# crea una lista con las claves de c_dict
c_keys = list(c_dict.keys())
for key in c_query_dict:
    N_cases_check = factor * len(c_query_dict[key]) # número de sentencias no relevantes a añadir
    # baraja s_keys para no tomar siempre las mismas de relleno
    random.shuffle(c_keys)
    while len(c_query_dict[key]) < N_cases_check:
        for c in c_keys:
            if c not in c_query_dict[key]:
                c_query_dict[key].append(c)
                if len(c_query_dict[key]) == N_cases_check:
                    break


# baraja los elementos de la lista de valores de cada entrada de c_query_dict
# para que no estén siempre al principio las sentencias relevantes
for key in c_query_dict:
    random.shuffle(c_query_dict[key])


In [ ]:

# Generamos el prompt para los status
query = "AILA_Q5" # query a buscar
n_relevant = len(s_relevance_dict[query]) # número de sentencias relevantes


ask_for_query = "\n".join(f"Statute index: {statute}\n{s_dict[statute]}" 
                       for statute in s_query_dict[query])

# print(ask_for_query)

prompt_template = f"""
<s>[INST] Given the following information about statutes from Indian law, please indicate which ones 
are the {n_relevant} most relevant for the described situation that had led to filing a case in an Indian court of law. 
Statutes are considered relevant to a situation if they discusses a situation similar to that in the query, 
as judged by law experts.


Statutes:
{ask_for_query}

Situation: {query_dict[query]}
[/INST]
"""
print(prompt_template)




In [ ]:
# Generamos el prompt para los status
query = "AILA_Q5" # query a buscar
n_relevant = len(c_relevance_dict[query]) # número de sentencias relevantes


ask_for_query = "\n".join(f"Casedoc index: {case}\n{c_dict[case]}" 
                       for case in c_query_dict[query])

# print(ask_for_query)

prompt_template = f"""
<s>[INST] Given the following information about cases from Indian law, please indicate which ones 
are the {n_relevant} most relevant for the described situation that had led to filing a case in an Indian court of law. 
Cases are considered relevant to a situation if they discusses a situation similar to that in the query, 
as judged by law experts.


Cases:
{ask_for_query}

Situation: {query_dict[query]}
[/INST]
"""
print(prompt_template)

In [ ]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are an expert in law."},
    {"role": "user",  "content": prompt_template},
]
outputs = pipeline(
    messages,
    max_new_tokens=5000,
)
print(outputs[0]["generated_text"][-1])

In [ ]:
from openai import OpenAI
import os


# Función que recibe una cadena con un nombre de modelo y un prompt, y devuelve el resultado
# de la conversación con el modelo
def ask_OpenAI(modelo, prompt):
    
    clave_api = os.getenv('OPENAI_API_KEY') 

    client = OpenAI(
        api_key=clave_api
    )

    chat_completion = client.chat.completions.create(
        messages=[
            {'role': 'system', 'content': 'You are an expert in Indian law and legal cases.'},
            {'role': 'user', 'content': prompt}
        ],
        #model="gpt-3.5-turbo",
        model=modelo,
        temperature=0.7 # más cercano a 0 es más determinista, más cercano a 1 es creativo
    )
    # retorno de la función get_chat_completion
    return chat_completion.choices[0].message.content


In [ ]:
modelo = "gpt-4o" 

respuesta = ask_OpenAI(modelo, prompt_template)
print(respuesta)

In [ ]:


file_path = "../resultados"

# para cada sentencia relevante de la query busca si está contenida en el resultado
# de la conversación, y calcula el número de aciertos
modelo = "gpt-4o" 

# recorre todas las queries y hace la consulta con un conjunto de sentencias relevantes y no relevantes
for query in query_dict:
    # imprime la query que se está procesando
    print("Procesando la query ", query)
    # imprime las sentencias relevantes de la query
    print("Sentencias relevantes de la query ", s_relevance_dict[query])
    # imprime las sentencias relevantes y no relevantes de la query
    print("Sentencias relevantes y no relevantes de la query ", s_query_dict[query])

    n_relevant = len(s_relevance_dict[query]) # número de sentencias relevantes
    ask_for_query = "\n".join(f"Statute index: {statute}\n{s_dict[statute]}" 
                        for statute in s_query_dict[query])
    # generamos el prompt
    prompt_template = f"""
    <s>[INST] Given the following information about statutes from Indian law, please indicate which ones 
    are the {n_relevant} most relevant for the described situation that had led to filing a case in an Indian court of law. 
    Statutes are considered relevant to a situation if they discusses a situation similar to that in the query, 
    as judged by law experts.


    Statutes:
    {ask_for_query}

    Situation: {query_dict[query]}
    [/INST]
    """
    respuesta = ask_OpenAI(modelo, prompt_template)
    # calculamos la precisión
    aciertos = 0
    for s in s_relevance_dict[query]:
        if s in respuesta:
            aciertos += 1
    # calcula el porcentaje de aciertos sobre el número de sentencias relevantes y no relevantes de la query
    precision = aciertos / len(s_relevance_dict[query]) * 100

    # crea un fichero con el resultado de la consulta
    resultado = f"""
    CONSULTA REALIZADA:
    {prompt_template}
    ----------------------------------------------------

    RESPUESTA DE {modelo}:
    {respuesta}
    ----------------------------------------------------

    LISTA DE SENTENCIAS RELEVANTES PARA LA QUERY {query}:
    {str(s_relevance_dict[query])}
    ----------------------------------------------------

    LISTA DE SENTENCIAS RELEVANTES Y NO RELEVANTES PARA LA QUERY {query}:
    {str(s_query_dict[query])}
    ----------------------------------------------------

    PRECISIÓN {query}: {precision}%
    """

    file_name = file_path + f"/chat_completion_{query}.txt"
    # Guarda el resultado en un fichero, y crealo si no existe o truncalo si existe
    with open(file_name, "w") as file:
        file.write(resultado)
    # cierra el fichero
    file.close()
    print("Se ha procesado la query ", query)

In [ ]:


file_path = "../resultados/queries_Nfactor_cases"

# para cada caso relevante de la query busca si está contenid en el resultado
# de la conversación, y calcula el número de aciertos
modelo = "gpt-4o" 

# recorre todas las queries y hace la consulta con un conjunto de sentencias relevantes y no relevantes
for query in query_dict:
    # comprueba si existe el fichero con el resultado de la consulta
    if os.path.exists(file_path + f"/chat_completion_{query}.txt"):
        # si existe, pasa a la siguiente query
        print("Ya se ha procesado la query ", query)
        continue

    # imprime la query que se está procesando
    print("Procesando la query ", query)
    # imprime los casos relevantes de la query
    print("Casos relevantes de la query ", c_relevance_dict[query])
    # imprime las sentencias relevantes y no relevantes de la query
    print("Casos relevantes y no relevantes de la query ", c_query_dict[query])

    n_relevant = len(c_relevance_dict[query]) # número de casos relevantes
    ask_for_query = "\n".join(f"Case index: {casedoc}\n{c_dict[casedoc]}" 
                        for casedoc in c_query_dict[query])
    # generamos el prompt
    prompt_template = f"""
    <s>[INST] Given the following information about cases from Indian law, please indicate which ones 
    are the {n_relevant} most relevant for the described situation that had led to filing a case in an Indian court of law. 
    Cases are considered relevant to a situation if they discusses a situation similar to that in the query, 
    as judged by law experts.


    Cases:
    {ask_for_query}

    Situation: {query_dict[query]}
    [/INST]
    """
    respuesta = ask_OpenAI(modelo, prompt_template)
    # calculamos la precisión
    aciertos = 0
    for c in c_relevance_dict[query]:
        if c in respuesta:
            aciertos += 1
    # calcula el porcentaje de aciertos sobre el número de sentencias relevantes y no relevantes de la query
    precision = aciertos / len(c_relevance_dict[query]) * 100

    # crea un fichero con el resultado de la consulta
    resultado = f"""
    CONSULTA REALIZADA:
    {prompt_template}
    ----------------------------------------------------

    RESPUESTA DE {modelo}:
    {respuesta}
    ----------------------------------------------------

    LISTA DE SENTENCIAS RELEVANTES PARA LA QUERY {query}:
    {str(c_relevance_dict[query])}
    ----------------------------------------------------

    LISTA DE SENTENCIAS RELEVANTES Y NO RELEVANTES PARA LA QUERY {query}:
    {str(c_query_dict[query])}
    ----------------------------------------------------

    PRECISIÓN {query}: {precision}%
    """

    file_name = file_path + f"/chat_completion_{query}.txt"
    # Guarda el resultado en un fichero, y crealo si no existe o truncalo si existe
    with open(file_name, "w") as file:
        file.write(resultado)
    # cierra el fichero
    file.close()
    print("Se ha procesado la query ", query)

In [ ]:
file_path = "../resultados"
modelo = "gpt-4o" 


# recorre todas las queries, y para cada querie recorre todas las sentencias.
# imprimiendo por pantalla el par query-sentencia
s_relevance_calc_dict = {} # diccionario con las sentencias relevantes calculadas

for q in query_dict:
    s_relevance_calc_dict[q] = []
    for s in s_dict:
        prompt_template = f"""
        <s>[INST] Given the following information about statutes from Indian law, please indicate if the indicated
        statute is relevant for the described situation that had led to filing a case in an Indian court of law, answering
        with a yes or no. Statutes are considered relevant to a situation if they discusses a situation similar to that in the query,
        as judged by law experts.
        
        Statute: {s_dict[s]}

        Situation: {query_dict[q]}
        [/INST]
        """
        respuesta = ask_OpenAI(modelo, prompt_template)
        print(f"Query: {q} - Sentencia: {s} - Respuesta: {respuesta}")
        if "Yes" in respuesta:
            s_relevance_calc_dict[q].append(s)

# construye una tabla de resultados con las sentencias relevantes calculadas
# en la que cada línea tiene el nombre de la query, y separadas por coma
# las sentencias relevantes calculadas
table = "Leyes relevantes calculadas por la IA para cada query\n"
for q in s_relevance_calc_dict:
    table += f"{q},{','.join(s_relevance_calc_dict[q])}\n"
# imprime la tabla y la guarda en un fichero
print(table)
file_name = file_path + f"/leyes_relevantes_calculadas_para_todas_queries.txt"
with open(file_name, "w") as file:
    file.write(table)
file.close()

# construye una tabla de resultados con todas las queries, en el que la primera columna
# es el nombre de la query, la segunda columna el número de aciertos positivos, es decir,
# aquellos statutes que se encuentran en s_relevance_dict y en s_relevance_calc_dict
# la tercera columna es el número de aciertos negativos, es decir, aquellos statutes que
# no se encuentran en s_relevance_dict y tampoco en s_relevance_calc_dict, la cuarta columna
# es el número de falsos positivos, es decir, aquellos statutes que se encuentran en
# s_relevance_calc_dict pero no en s_relevance_dict, y la quinta columna es el número de
# falsos negativos, es decir, aquellos statutes que se encuentran en s_relevance_dict
# pero no en s_relevance_calc_dict.
table = "Comparación de resultados para todas las queries\n"
table += "Query,Aciertos positivos,Aciertos negativos,Falsos positivos,Falsos negativos\n"
for q in s_relevance_dict:
    aciertos_positivos = len(set(s_relevance_dict[q]) & set(s_relevance_calc_dict[q]))
    falsos_positivos = len(set(s_relevance_calc_dict[q]) - set(s_relevance_dict[q]))
    falsos_negativos = len(set(s_relevance_dict[q]) - set(s_relevance_calc_dict[q]))
    aciertos_negativos = len(s_dict) - aciertos_positivos - falsos_positivos - falsos_negativos
    table += f"{q},{aciertos_positivos},{aciertos_negativos},{falsos_positivos},{falsos_negativos}\n"

# imprime la tabla y la guarda en un fichero
print(table)
file_name = file_path + f"/comparacion_resultados_statutes_para_todas_queries.txt"
with open(file_name, "w") as file:
    file.write(table)
file.close()




In [ ]:
file_path = "../resultados"
modelo = "gpt-4o" 


# recorre todas las queries, y para cada querie recorre todos los casedocs.
# imprimiendo por pantalla el par query-casedoc
c_relevance_calc_dict = {} # diccionario con las sentencias relevantes calculadas

for q in query_dict:
    c_relevance_calc_dict[q] = []
    for c in c_dict:
        prompt_template = f"""
        <s>[INST] Given the following information about cases from Indian law, please indicate if the indicated
        case is relevant for the described situation that had led to filing a case in an Indian court of law, answering
        with a yes or no. Cases are considered relevant to a situation if they discusses a situation similar to that in the query,
        as judged by law experts.
        
        Case: {c_dict[c]}

        Situation: {query_dict[q]}
        [/INST]
        """
        respuesta = ask_OpenAI(modelo, prompt_template)
        # genera una cadena con la consulta y la respuesta
        consultaYrespuesta = f"Query: {q} - Casedoc: {c} - Respuesta: {respuesta}"
        print(consultaYrespuesta)
        if "Yes" in respuesta:
            c_relevance_calc_dict[q].append(c)
        # guarda la respuesta en un fichero
        file_name = file_path + f"/queries_casos/respuesta_{q}_{c}.txt"
        with open(file_name, "w") as file:
            file.write(consultaYrespuesta)


# construye una tabla de resultados con las casos relevantes calculadas
# en la que cada línea tiene el nombre de la query, y separadas por coma
# los casos relevantes calculados
table = "Casos relevantes calculados por la IA para cada query\n"
for q in c_relevance_calc_dict:
    table += f"{q},{','.join(c_relevance_calc_dict[q])}\n"
# imprime la tabla y la guarda en un fichero
print(table)
file_name = file_path + f"/casos_relevantes_calculados_para_todas_queries.txt"
with open(file_name, "w") as file:
    file.write(table)
file.close()




In [ ]:
import os
file_path = "../resultados"
result_path = file_path + f"/queries_casos/"

# lee todos los ficheros de result_path y los imprime por pantalla y construye una tabla de resultados
# la query de referencia, el casedoc de referencia y la respuesta. La respuesta es Yes si el casedoc
# es relevante para la query, y No si no lo es. En la tabla poner un 1 si la respuesta es Yes y un 0 si
# la respuesta es No. El nombre de la query y el casedoc se obtiene del nombre del fichero. El de la query
# es el que se encuentra entre dos _ y el del casedoc es el que se encuentra entre el segundo _ y el .txt

# creo un diccionario de casos relevantes para cada query a partir de los ficheros de resultados
c_relevance_calc_dict = {} # diccionario con las sentencias relevantes calculadas
table = ""
for file_name in os.listdir(result_path):
    with open(result_path + file_name, "r") as file:
        content = file.read()
        # obtiene la query y el casedoc del nombre del fichero
        query = file_name.split("_")[2]
        casedoc = file_name.split("_")[3].split(".")[0]
        # obtiene la respuesta
        respuesta = "1" if "Yes" in content else "0"
        table += f"AILA_{query} Q0 {casedoc} {respuesta}\n"
        if respuesta == "1":
            query = f"AILA_{query}"
            if query in c_relevance_calc_dict:
                c_relevance_calc_dict[query].append(casedoc)
            else:
                c_relevance_calc_dict[query] = [casedoc]
        # print(f"Query: {query} - Casedoc: {casedoc} - Respuesta: {respuesta}")  
# imprime la tabla y la guarda en un fichero
# print(table)
file_name = file_path + f"/comparacion_resultados_casedocs_para_todas_queries.txt"
with open(file_name, "w") as file:
    file.write(table)
file.close()

# construye una tabla de resultados con todas las queries, en el que la primera columna
# es el nombre de la query, la segunda columna el número de aciertos positivos, es decir,
# aquellos casos que se encuentran en c_relevance_dict y en c_relevance_calc_dict
# la tercera columna es el número de aciertos negativos, es decir, aquellos casos que
# no se encuentran en c_relevance_dict y tampoco en c_relevance_calc_dict, la cuarta columna
# es el número de falsos positivos, es decir, aquellos statutes que se encuentran en
# c_relevance_calc_dict pero no en c_relevance_dict, y la quinta columna es el número de
# falsos negativos, es decir, aquellos statutes que se encuentran en c_relevance_dict
# pero no en c_relevance_calc_dict.
table = "Comparación de resultados para todas las queries\n"
table += "Query,Aciertos positivos,Aciertos negativos,Falsos positivos,Falsos negativos\n"
for q in c_relevance_calc_dict:
    print("solucion: ",c_relevance_dict[q])
    print("calculado: ",c_relevance_calc_dict[q])
    aciertos_positivos = len(set(c_relevance_dict[q]) & set(c_relevance_calc_dict[q]))
    falsos_positivos = len(set(c_relevance_calc_dict[q]) - set(c_relevance_dict[q]))
    falsos_negativos = len(set(c_relevance_dict[q]) - set(c_relevance_calc_dict[q]))
    aciertos_negativos = len(c_dict) - aciertos_positivos - falsos_positivos - falsos_negativos
    table += f"{q},{aciertos_positivos},{aciertos_negativos},{falsos_positivos},{falsos_negativos}\n"

# imprime la tabla y la guarda en un fichero
print(table)
file_name = file_path + f"/tabla_resultados_casedocs_para_todas_queries.txt"
with open(file_name, "w") as file:
    file.write(table)
file.close()

In [ ]:

file_path = "../resultados/exp5_statutes"
modelo = "gpt-4o"
lista_queries_test = ["AILA_Q5","AILA_Q12","AILA_Q17","AILA_Q28"]
# para cada query de la lista de queries de test imprime el nombre por pantalla
for query in lista_queries_test:
    # agrega el nombre de la query al de file_path
    file_path += f"/{query}"
    # crea el directorio si no existe
    if not os.path.exists(file_path):
        os.makedirs(file_path)
    # imprime los statutes relevantes para la query de ejemplo
    print("Statutes relevantes para la query de ejemplo ",query)
    print(s_relevance_dict[query])
    grupo_filtro1 =[]  # grupo de statutes seleccionadas de cada grupo inicial
    grupo_filtro2 =[]  # grupo de statutes seleccionadas del grupo_filtro1

    # recorre todas las statutes y crea grupos de 10 statutes y almacenalas
    # en un array de grupos
    # imprime los grupos de statutes
    #n = 10
    n = 4
    groups = [list(s_dict.keys())[i:i + n] for i in range(0, len(s_dict), n)]
    for i, group in enumerate(groups):
        print(f"Grupo {i}: {group}")
        ask_for_query = "\n".join(f"Statute index: {s}\n{s_dict[s]}" 
                                for s in group) 
        # generamos el prompt
        # n_relevant = 3 # número de sentencias relevantes que cogemos en este experimentos
        n_relevant = 2 # número de sentencias relevantes que cogemos en este experimentos
        prompt_template = f"""
        <s>[INST] Given the following information about statutes from Indian law, please indicate which ones 
        are the {n_relevant} most relevant for the described situation that had led to filing a case in an Indian court of law. 
        Statutes are considered relevant to a situation if they discusses a situation similar to that in the query, 
        as judged by law experts.


        Statutes:
        {ask_for_query}

        Situation: {query_dict[query]}
        [/INST]
        """
        respuesta = ask_OpenAI(modelo, prompt_template)
        fichero = file_path + f"/respuesta_query_{query}_grupo_{i}.txt"
        # graba la respuesta al fichero
        with open(fichero, "w") as file:
            file.write(respuesta)
        # imprime la respuesta    
        # print(respuesta)
        # para cada statute del grupo comprueba si está en la respuesta y si es así lo añade al grupo_filtro1
        for s in group:
            if s in respuesta:
                print(f"Statute {s} is relevant del grupo {i}")
                grupo_filtro1.append(s)
    # baraja el grupo de statutes seleccionadas
    random.shuffle(grupo_filtro1)
    # a partir del grupo_filtro1 crea grupos de 10 statutes y los imprime
    # n = 10  
    n = 3
    groups = [grupo_filtro1[i:i + n] for i in range(0, len(grupo_filtro1), n)]
    for i, group in enumerate(groups):
        print(f"Grupo {i}: {group}")
        ask_for_query = "\n".join(f"Statute index: {s}\n{s_dict[s]}" 
                                for s in group) 
        # generamos el prompt
        # n_relevant = 2 # número de sentencias relevantes que cogemos en este experimentos
        n_relevant = 1 # número de sentencias relevantes que cogemos en este experimentos
        prompt_template = f"""
        <s>[INST] Given the following information about statutes from Indian law, please indicate which ones
        are the {n_relevant} most relevant for the described situation that had led to filing a case in an Indian court of law.
        Statutes are considered relevant to a situation if they discusses a situation similar to that in the query,
        as judged by law experts.

        
        Statutes:
        {ask_for_query}

        Situation: {query_dict[query]}
        [/INST]
        """
        respuesta = ask_OpenAI(modelo, prompt_template)
        fichero = file_path + f"/respuesta_query_{query}_grupo_filtro2_{i}.txt"
        # graba la respuesta al fichero
        with open(fichero, "w") as file:
            file.write(respuesta)
        # imprime la respuesta
        # print(respuesta)
        # para cada statute del grupo comprueba si está en la respuesta y si es así lo añade al grupo_filtro2
        for s in group:
            if s in respuesta:
                print(f"Statute {s} is relevant del grupo {i}")
                grupo_filtro2.append(s)
        

    # baraja el grupo de statutes seleccionadas
    random.shuffle(grupo_filtro2)
    n_relevant = 5 # escogemos con una última consulta 5 statutes más relevantes
    ask_for_query = "\n".join(f"Statute index: {s}\n{s_dict[s]}" 
                            for s in grupo_filtro2) 
    prompt_template = f"""
        <s>[INST] Given the following information about statutes from Indian law, please indicate which ones
        are the {n_relevant} most relevant for the described situation that had led to filing a case in an Indian court of law.
        Statutes are considered relevant to a situation if they discusses a situation similar to that in the query,
        as judged by law experts.

        
        Statutes:
        {ask_for_query}

        Situation: {query_dict[query]}
        [/INST]
        """
    respuesta = ask_OpenAI(modelo, prompt_template)
    fichero = file_path + f"/respuesta_query_{query}_solucionPropuesta.txt"
    # graba la respuesta al fichero
    with open(fichero, "w") as file:
        file.write(respuesta)
    solucion_propuesta =[]
    # para cada statute del grupo comprueba si está en la respuesta y si es así lo añade solucion_propuesta
    for s in grupo_filtro2:
        if s in respuesta:
            print(f"Statute {s} is relevant")
            solucion_propuesta.append(s)
    print("Statutes relevantes para la query de ejemplo ",query)
    print(s_relevance_dict[query])
    print("Statutes propuestas como solucion: ",solucion_propuesta)
    # calcula el número de aciertos
    aciertos = len(set(s_relevance_dict[query]) & set(solucion_propuesta))
    print("Número de aciertos: ",aciertos)
    # calcula la precisión
    precision = aciertos / len(s_relevance_dict[query]) * 100
    print("Precisión: ",precision)
    # crea un fichero con los resultados que incluya la query, las statutes relevantes, las statutes propuestas
    # como solución y la precisión
    resultado = f"""
    Query: {query}
    Statutes relevantes: {s_relevance_dict[query]}
    Statutes propuestas como solución: {solucion_propuesta}
    Precisión: {precision}
    """
    fichero = file_path + f"/resultado_query_{query}.txt"
    # graba el resultado en el fichero
    with open(fichero, "w") as file:
        file.write(resultado)
    

    




In [6]:
import openai
import os
import tiktoken # módulo con funciones para trabajar con tokens
# cojo el API_KEY de la variable de entorno
mi_api_key = os.getenv('OPENAI_API_KEY') # configurada en activate del entorno $HOME/ULL-env/bin/activate

# Configurar API Key de OpenAI
openai_client = openai.OpenAI(api_key=mi_api_key)  # Nuevo cliente para OpenAI



# los modelos usados de OpenAI nbo soportan mas de 8192 tokens
# Como los textos no son muy largos (los casedocs y las statutes) optamos por trucar
# los textos a 8192 tokens. Si fueran más largos habría que dividirlos en trozos
# de 8192 tokens y hacer varias consultas, y luego unir los resultados. Existen varias
# estrategias como sacar el vector promedio de los embeddings de cada trozo. En este caso
# no es necesario porque los textos no son muy largos.

#modelo_openAI="text-embedding-3-small"
modelo_openAI="text-embedding-3-large"
#modelo_openAI="text-embedding-ada-002"

# Función para obtener embeddings usando la API moderna de OpenAI
# embeddings avanzados de OpenAI text-embedding-3-large y text-embedding-3-small
# Requiere que esté definida modelo_openAI

def get_embedding_OpenAI(text, model=modelo_openAI):
    try:
        encoding = tiktoken.encoding_for_model(model)
        MAX_TOKENS = 8192  # Límite del modelo
        
        tokens = encoding.encode(text)  # Tokenizar texto
        
        if len(tokens) > MAX_TOKENS:
            print(f"Warning: Texto con {len(tokens)} tokens excede el límite ({MAX_TOKENS}). Se truncará.")
            tokens = tokens[:MAX_TOKENS]  # Truncar al límite permitido
            text = encoding.decode(tokens)  # Convertir tokens truncados a texto

        response = openai_client.embeddings.create(input=[text], model=model)
        return response.data[0].embedding
    except openai.OpenAIError as e:
        print(f"Error en OpenAI al obtener embeddings del texto: {e}")
        return None
    except Exception as e:
        print(f"Error desconocido: {e}")
        return None


In [ ]:
import chromadb
from sentence_transformers import SentenceTransformer

# Inicializar ChromaDB en memoria (sin persistencia en disco)
# chroma_client = chromadb.EphemeralClient()
# si se quiere hacer con persistencia en disco

# descomentar la siguiente linea para embeddings de OpenAI
#embedding_name = "OpenAI" # nombre del modelo de embeddings OpenAI

# descomentar las siguientes 3 lineas para embeddings de HF model all-MiniLM-L6-v2
#embedding_name = "HF_MiniLM" # nombre del modelo de embeddings MiniLM
#nombre_modelo = "all-MiniLM-L6-v2"  # mas ligero pero menos preciso
#model = SentenceTransformer(nombre_modelo)  

# descomentar las siguientes 3 lineas para embeddings de HF model all-mpnet-base-v2
#embedding_name = "HF_mpnet" # nombre del modelo de embeddings mpnet
#nombre_modelo = "sentence-transformers/all-mpnet-base-v2"  # mas pesado pero mas preciso
#model = SentenceTransformer(nombre_modelo)  

# descomentar las siguientes 3 lineas para embeddings de HF model all-mpnet-base-v2- fine tunning
embedding_name = "HF_mpnet_finetunning" # nombre del modelo de embeddings mpnet con fine tunning
nombre_modelo = "../modelos_fine_tunning/sentence-transformers/all-mpnet-base-v2_finetunning"  # con finetunning del mpnet
model = SentenceTransformer(nombre_modelo)  





path_DB = "../chroma_db_"+embedding_name
chroma_client = chromadb.PersistentClient(path=path_DB)  
collection = chroma_client.get_or_create_collection(name=embedding_name)

# separa los statutes por un lado los identificadores, y por otros los textos
statutes = list(s_dict.keys())
texts = list(s_dict.values())

# Generar y almacenar embeddings en ChromaDB
# generar embeddings si el modelo es HF_MiniLM o HF_mpnet

if embedding_name == "HF_MiniLM" or embedding_name == "HF_mpnet" or embedding_name == "HF_mpnet_finetunning":
    # si no se recorre en un bucle como con OpenAI se procesan todos los textos
    # de una vez aprovechando la capacidad de procesamiento paralelo
    embeddings = model.encode(texts).tolist()  # Obtener embeddings de cada texto
# sino generar embeddings con OpenAI
elif embedding_name == "OpenAI":    
    embeddings = [get_embedding_OpenAI(text) for text in texts]  # Obtener embeddings de cada texto

collection.add(documents=texts, embeddings=embeddings, ids=statutes) # lo añadimos con su id de statute

print("Embeddings almacenados correctamente en ChromaDB para embeddings",embedding_name)




In [ ]:
import chromadb
from sentence_transformers import SentenceTransformer

# Inicializar ChromaDB en memoria (sin persistencia en disco)
# chroma_client = chromadb.EphemeralClient()
# si se quiere hacer con persistencia en disco

# descomentar la siguiente linea para embeddings de OpenAI
embedding_name = "OpenAI" # nombre del modelo de embeddings OpenAI

# descomentar las siguientes 3 lineas para embeddings de HF model all-MiniLM-L6-v2
#embedding_name = "HF_MiniLM" # nombre del modelo de embeddings MiniLM
#nombre_modelo = "all-MiniLM-L6-v2"  # mas ligero pero menos preciso
#model = SentenceTransformer(nombre_modelo)  

# descomentar las siguientes 3 lineas para embeddings de HF model all-mpnet-base-v2
#embedding_name = "HF_mpnet" # nombre del modelo de embeddings mpnet
#nombre_modelo = "sentence-transformers/all-mpnet-base-v2"  # mas pesado pero mas preciso
#model = SentenceTransformer(nombre_modelo)  

path_DB = "../chroma_db_casedocs_"+embedding_name
#path_DB = "./chroma_db"
chroma_client = chromadb.PersistentClient(path=path_DB)  
collection = chroma_client.get_or_create_collection(name=embedding_name)
print("Colección de embeddings de casedocs creada correctamente")

# separa los statutes por un lado los identificadores, y por otros los textos
casedocs = list(c_dict.keys())
texts = list(c_dict.values())

# Generar y almacenar embeddings en ChromaDB
# generar embeddings si el modelo es HF_MiniLM o HF_mpnet
embeddings = []
if embedding_name == "HF_MiniLM" or embedding_name == "HF_mpnet":
    # si no se recorre en un bucle como con OpenAI se procesan todos los textos
    # de una vez aprovechando la capacidad de procesamiento paralelo
    embeddings = model.encode(texts).tolist()  # Obtener embeddings de cada texto
# sino generar embeddings con OpenAI
elif embedding_name == "OpenAI":    
    # crea una lista vacia de entradas del diccionario que no se han podido procesar
    # para eliminarlas del diccionario
    c_dict_sin_procesar = []
    for c in c_dict:
        # imprime la clave del casedoc que se está procesando        
        # print(f"Calculando embedding de casedoc: {c}\n")
        # calcula el embedding de cada valor del diccionario y lo añades
        # a la lista de embeddings
        emb = get_embedding_OpenAI(c_dict[c])  # Obtener embeddings de cada texto
        if emb is None:
            print(f"Error al calcular el embedding de casedoc: {c}\n")
            # si no se ha podido calcular el embedding añade la clave a la lista
            # de casedocs sin procesar
            c_dict_sin_procesar.append(c)
        else:   
            embeddings.append(emb)
            #print(f"Embedding de casedoc {c} calculado correctamente. Tamaño actual {len(embeddings)}\n")
    # elimina las claves sin procesar del diccionario
    for c in c_dict_sin_procesar:
        del c_dict[c]
    # volver a generar las listas porque pueden haberse eliminados algunos casedocs
    casedocs = list(c_dict.keys())
    texts = list(c_dict.values())
    # la siguiente linea era para no hacerlo en un bucle e imprimiendo los casos   
    #embeddings = [get_embedding_OpenAI(text) for text in texts]  # Obtener embeddings de cada texto

collection.add(documents=texts, embeddings=embeddings, ids=casedocs) # lo añadimos con su id de statute

print("Embeddings de casedocs almacenados correctamente en ChromaDB para embeddings",embedding_name)

In [ ]:
import chromadb
from sentence_transformers import SentenceTransformer

# Inicializar ChromaDB en memoria (sin persistencia en disco)
# chroma_client = chromadb.EphemeralClient()
# si se quiere hacer con persistencia en disco
# hago la consulta de las n_cercanas
n_cercanas = 2500
#embedding_name = "OpenAI" # nombre del modelo de embeddings, por si se quieren probar varios
#embedding_name = "HF_MiniLM" # nombre del modelo de embeddings MiniLM
embedding_name = "HF_mpnet" # nombre del modelo de embeddings mpnet

# para OpenAI no es necesario inicializar el modelo
if embedding_name == "HF_MiniLM":
    nombre_modelo = "all-MiniLM-L6-v2"  # mas ligero pero menos preciso
    model = SentenceTransformer(nombre_modelo)

if embedding_name == "HF_mpnet":
    nombre_modelo = "sentence-transformers/all-mpnet-base-v2"  # mas pesado pero mas preciso
    model = SentenceTransformer(nombre_modelo)


path_DB = "../kaggle_AILA/chroma_db_casedocs_"+embedding_name
chroma_client = chromadb.PersistentClient(path=path_DB)  

collection = chroma_client.get_or_create_collection(name=embedding_name)

fichero_resultados = f"""../resultados/embeddings_casedocs/resultados_embeddings_{embedding_name}_{n_cercanas}_cercanas.txt"""
# abre el fichero de resultados de escritura y escribe la cabecera

with open(fichero_resultados, "w") as file:
    file.write(f"""Resultados de las consultas con embeddings casedocs de {embedding_name} de las {n_cercanas} mas cercanas:""")
    file.write("\n--------------------------------------------------------\n")
    file.write("\n")

# inicializa la precisión acumulada
precision_acumulada = 0
# recorre todas las querys y para cada query hace una consulta con el texto de la query
# y obtiene las n_cercanas casedocs
# en cada query se compara las casedocs obtenidas con las que se encuentran en s_relevance_dict
# y se calcula el número de aciertos
for query in query_dict:
    # obtener el texto de la query del diccionario
    query_text=query_dict[query]
    # Calculo el embedding del texto de la query
    if embedding_name == "HF_MiniLM" or embedding_name == "HF_mpnet":
        query_embedding = model.encode([query_text]).tolist()[0]
    elif embedding_name == "OpenAI":
        query_embedding = get_embedding_OpenAI(query_text)

    results = collection.query(query_embeddings=[query_embedding], n_results=n_cercanas)

    aciertos = 0
    for casedoc in c_relevance_dict[query]:
        if casedoc in results['ids'][0]:
            aciertos += 1
    # calculo la precisión
    precision = aciertos / len(c_relevance_dict[query]) * 100
    precision_acumulada += precision

    # añadimos a un fichero los resultados
    resultado = f"""
    Query: {query}
    Casos relevantes: {c_relevance_dict[query]}
    {n_cercanas} casos más cercanos: {results['ids'][0]}
    Número de aciertos: {aciertos}
    Precisión: {precision}
    """
    with open(fichero_resultados, "a") as file:
        file.write(resultado)


    # si quisieramos imprimir los resultados de los statutes con sus textos debemos
    # tener en cuenta que es un diccionario en el que los nombres de statues están en el 'ids'
    # y los textos en 'documents' 
    # for i in range(len(results["documents"][0])):
    #     print(f"{i+1}. Statute: {results['ids'][0][i]} - Texto: {results['documents'][0][i]}")


# calculo la precisión media
precision_media = precision_acumulada / len(query_dict)
# añado al fichero de resultados la precisión media
with open(fichero_resultados, "a") as file:
    file.write(f"\n*************************************\n")
    file.write(f"Precision media: {precision_media}")
# imprimo por consola el fichero
with open(fichero_resultados, "r") as file:
    print(file.read())
#cierro el fichero de resultados
file.close()


In [ ]:
import chromadb

# Inicializar ChromaDB en memoria (sin persistencia en disco)
# chroma_client = chromadb.EphemeralClient()
# si se quiere hacer con persistencia en disco
# hago la consulta de las n_cercanas
n_cercanas = 197
#embedding_name = "OpenAI" # nombre del modelo de embeddings, por si se quieren probar varios
#embedding_name = "HF_MiniLM" # nombre del modelo de embeddings MiniLM
#embedding_name = "HF_mpnet" # nombre del modelo de embeddings mpnet
embedding_name = "HF_mpnet_finetunning" # nombre del modelo de embeddings mpnet re-entrenado

# para OpenAI no es necesario inicializar el modelo
if embedding_name == "HF_MiniLM":
    nombre_modelo = "all-MiniLM-L6-v2"  # mas ligero pero menos preciso
    model = SentenceTransformer(nombre_modelo)

if embedding_name == "HF_mpnet":
    nombre_modelo = "sentence-transformers/all-mpnet-base-v2"  # mas pesado pero mas preciso
    model = SentenceTransformer(nombre_modelo)

if embedding_name == "HF_mpnet_finetunning":
    nombre_modelo = "../modelos_fine_tunning/sentence-transformers/all-mpnet-base-v2_finetunning"  # mas pesado pero mas preciso
    model = SentenceTransformer(nombre_modelo)


path_DB = "../chroma_db_"+embedding_name
chroma_client = chromadb.PersistentClient(path=path_DB)  

collection = chroma_client.get_or_create_collection(name=embedding_name)

fichero_resultados = f"""../resultados/embeddings/resultados_embeddings_{embedding_name}_{n_cercanas}_cercanas.txt"""
# abre el fichero de resultados de escritura y escribe la cabecera

with open(fichero_resultados, "w") as file:
    file.write(f"""Resultados de las consultas con embeddings de {embedding_name} de las {n_cercanas} mas cercanas:""")
    file.write("\n--------------------------------------------------------\n")
    file.write("\n")

# inicializa la precisión acumulada
precision_acumulada = 0
# recorre todas las querys y para cada query hace una consulta con el texto de la query
# y obtiene las n_cercanas statutes
# en cada query se compara las statutes obtenidas con las que se encuentran en s_relevance_dict
# y se calcula el número de aciertos
for query in query_dict:
    # obtener el texto de la query del diccionario
    query_text=query_dict[query]
    # Calculo el embedding del texto de la query
    if embedding_name == "HF_MiniLM" or embedding_name == "HF_mpnet" or embedding_name == "HF_mpnet_finetunning":
        query_embedding = model.encode([query_text]).tolist()[0]
    elif embedding_name == "OpenAI":
        query_embedding = get_embedding_OpenAI(query_text)

    results = collection.query(query_embeddings=[query_embedding], n_results=n_cercanas)

    aciertos = 0
    for statute in s_relevance_dict[query]:
        if statute in results['ids'][0]:
            aciertos += 1
    # calculo la precisión
    precision = aciertos / len(s_relevance_dict[query]) * 100
    precision_acumulada += precision

    # añadimos a un fichero los resultados
    resultado = f"""
    Query: {query}
    Statutes relevantes: {s_relevance_dict[query]}
    {n_cercanas} Statutes más cercanos: {results['ids'][0]}
    Número de aciertos: {aciertos}
    Precisión: {precision}
    """
    with open(fichero_resultados, "a") as file:
        file.write(resultado)


    # si quisieramos imprimir los resultados de los statutes con sus textos debemos
    # tener en cuenta que es un diccionario en el que los nombres de statues están en el 'ids'
    # y los textos en 'documents' 
    # for i in range(len(results["documents"][0])):
    #     print(f"{i+1}. Statute: {results['ids'][0][i]} - Texto: {results['documents'][0][i]}")


# calculo la precisión media
precision_media = precision_acumulada / len(query_dict)
# añado al fichero de resultados la precisión media
with open(fichero_resultados, "a") as file:
    file.write(f"\n*************************************\n")
    file.write(f"Precision media: {precision_media}")
# imprimo por consola el fichero
with open(fichero_resultados, "r") as file:
    print(file.read())
#cierro el fichero de resultados
file.close()

In [ ]:
import pickle
# recorre todos los casedocs, y para cada casedoc recorre todas las queries
# si el casedoc es relevante para la query crea una terna con el texto del casedoc,
# el texto de la query y un 1 si es relevante o un 0 si no es relevante. Añade la terna
# a una lista de ternas. Al final crea un fichero con las ternas.
file_path = "../dataset_entrenamiento"
file_name = file_path + "/tripletas_casedocs_queries.txt"

dataset_t = []
for c in c_dict:
    for q in query_dict:
        relevante = 1 if c in c_relevance_dict[q] else 0
        # crea una tupla de la forma (c_dict[c], query_dict[q], relevante)
        terna = (c_dict[c], query_dict[q], relevante)
        # añade la terna a la lista de ternas
        dataset_t.append(terna)
        
# crea un dataset de entrenamiento equilibrado con el mismo número de ternas
# relevantes que no relevantes a partir del diccionario c_relevance_dict
# y el diccionario c_dict. Para cada casedoc relevante se selecciona un casedoc
# no relevante al azar y crea una terna con el texto del casedoc, el texto de la query
# y un 1 si es relevante o un 0 si no es relevante. Añade la terna a una lista de ternas.
dataset_eq_t = []
for q in query_dict:
    for c in c_relevance_dict[q]:
        relevante = 1
        # selecciona un casedoc no relevante al azar
        c_no_relevante = random.choice(list(set(c_dict.keys()) - set(c_relevance_dict[q])))
        # crea una tupla de la forma (c_dict[c_no_relevante], query_dict[q], 0)
        terna = (c_dict[c_no_relevante], query_dict[q], 0)
        # añade la terna a la lista de ternas
        dataset_eq_t.append(terna)
        # crea una tupla de la forma (c_dict[c], query_dict[q], 1)
        terna = (c_dict[c], query_dict[q], 1)
        # añade la terna a la lista de ternas
        dataset_eq_t.append(terna)

# baraja el dataset de ternas
random.shuffle(dataset_t)
# baraja el dataset de ternas equilibrado
#random.shuffle(dataset_eq_t)
# imprime las dimensiones de los datasets y la forma que tienen
print(f"Dimensiones del dataset de ternas: {len(dataset_t)}")
print(f"Dimensiones del dataset de ternas equilibrado: {len(dataset_eq_t)}")
# imprime las primeras ternas del data set equilibrado
for i in range(5):
    print(dataset_eq_t[i])
# salva a disco cada dataset de ternas para que puedan ser recuperadas en el mismo formato
# para futuros entrenamientos con pickle
file_name = file_path + "/tripletas_casedocs_queries.pkl"
with open(file_name, "wb") as file:
    pickle.dump(dataset_t, file)
file.close()
file_name = file_path + "/tripletas_casedocs_queries_equilibrado.pkl"
with open(file_name, "wb") as file:
    pickle.dump(dataset_eq_t, file)
file.close()




            
